## MLflow's Model Registry

In [ ]:
# ! mlflow ui --backend-store-uri sqlite:///mlflow.sqlite --port 5001

In [1]:
from mlflow.tracking import MlflowClient

# Create a connection to the tracking server
MLFLOW_TRACKING_URI = "sqlite:///mlflow.sqlite"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
# Create a tracking client
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Search all experiments
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716524700327, experiment_id='1', last_update_time=1716524700327, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [3]:
# create a new experiment
# client.create_experiment(name="my-cool-experiment")

Let's check the latest versions for the experiment with id `1`...

In [4]:
from mlflow.entities import ViewType

# Search all runs in the experiment
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f9a8c8bc049f4c8f8125e7c78ea0767d, rmse: 6.3007
run id: ce5fe048cb164a76bad49aebf2189c1b, rmse: 6.3007
run id: 871b3c5b08df47b1bdfd20f807e37e07, rmse: 6.3030
run id: a1eaff053ce44d9b885c41a5dca0f348, rmse: 6.3054
run id: 13eab02910f3418898829f7e848727cb, rmse: 6.3144


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [63]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [71]:
# xgboost model
run_id = "871b3c5b08df47b1bdfd20f807e37e07"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2024/05/24 06:22:19 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 6
Created version '6' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716531739435, current_stage='None', description=None, last_updated_timestamp=1716531739435, name='nyc-taxi-regressor', run_id='871b3c5b08df47b1bdfd20f807e37e07', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/871b3c5b08df47b1bdfd20f807e37e07/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [73]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Archived
version: 4, stage: Production
version: 6, stage: None


In [74]:
model_version = 6
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1716531739435, current_stage='Staging', description=None, last_updated_timestamp=1716531785430, name='nyc-taxi-regressor', run_id='871b3c5b08df47b1bdfd20f807e37e07', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/871b3c5b08df47b1bdfd20f807e37e07/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [75]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716531739435, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2024-05-24', last_updated_timestamp=1716531789271, name='nyc-taxi-regressor', run_id='871b3c5b08df47b1bdfd20f807e37e07', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/871b3c5b08df47b1bdfd20f807e37e07/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [76]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [77]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [81]:
# Load the data version
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

MlflowException: The following failures occurred while downloading one or more artifacts from /workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/871b3c5b08df47b1bdfd20f807e37e07/artifacts:
##### File preprocessor #####
[Errno 2] No such file or directory: '/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/871b3c5b08df47b1bdfd20f807e37e07/artifacts/preprocessor'

In [57]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [58]:
X_test = preprocess(df, dv)

In [59]:
target = "duration"
y_test = df[target].values

In [60]:
%time test_model(name=model_name, X_test=X_test, y_test=y_test, stage="Production")

OSError: No such file or directory: '/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f9a8c8bc049f4c8f8125e7c78ea0767d/artifacts/model/.'

In [61]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [06:20:02] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 19.8 s, sys: 23 ms, total: 19.8 s
Wall time: 5.3 s


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.252851825864611}

In [62]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1716531344425, current_stage='Production', description='The model version 4 was transitioned to Staging on 2024-05-24', last_updated_timestamp=1716531620559, name='nyc-taxi-regressor', run_id='f9a8c8bc049f4c8f8125e7c78ea0767d', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f9a8c8bc049f4c8f8125e7c78ea0767d/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>